In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
!pip install opencv-python

import cv2
from tensorflow.keras.models import model_from_json


Defaulting to user installation because normal site-packages is not writeable


In [10]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data

def check_dicom(filename):
    print('Load file {} ...\n'.format(filename))
    ds = pydicom.dcmread(filename) 
#     print(ds)
    
    # Validate DICOM fields as needed
    if ds.Modality != 'DX':
        print('Invalid Modality')
        return None
    
    if ds.PatientPosition not in ['AP', 'PA']:
        print('Invalid Position')
        return None
    
    if ds.BodyPartExamined != 'CHEST':
        print('Invalid Body part')
        return None
    
    patientAge = int(ds.PatientAge)
    if patientAge > 100:
        return None
    
    # Load image data
    img = ds.pixel_array
    return img


def preprocess_image(img, img_mean, img_std, img_size): 
    img = cv2.resize(img, (img_size[1], img_size[2]))
    img = (img - img_mean) / img_std
    img = np.stack([img, img, img], axis=-1)    
    img = np.reshape(img, img_size)
    
    return img


def load_model(model_path, weight_path):
    model_json = {}
    with open('my_model.json', 'r') as json_file:
        model_json = json_file.read()
    model = model_from_json(model_json)
    model.load_weights(weight_path)
    return model

def predict_image(model, img, thresh): 
    prediction = model.predict(img)
    print(prediction)
    return (prediction > thresh)[0][0]


In [8]:
model_path = 'my_model.json'
weight_path = 'xray_class_my_model.best.0.3.hdf5'
IMG_SIZE = (1, 224, 224, 3)
img_mean = 1
img_std = 1
thresh = 0.5

my_model = load_model(model_path, weight_path)

test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

for i in test_dicoms:
    img = check_dicom(i)
    if img is None:
        continue
    img_proc = preprocess_image(img, img_mean, img_std, IMG_SIZE)
    pred = predict_image(my_model, img_proc, thresh)
    print("Result", pred)
    print("----------")

Load file test1.dcm ...

(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.11129.5.5.110503645592756492463169821050252582267888
(0008, 0060) Modality                            CS: 'DX'
(0008, 1030) Study Description                   LO: 'No Finding'
(0010, 0020) Patient ID                          LO: '2'
(0010, 0040) Patient's Sex                       CS: 'M'
(0010, 1010) Patient's Age                       AS: '81'
(0018, 0015) Body Part Examined                  CS: 'CHEST'
(0018, 5100) Patient Position                    CS: 'PA'
(0020, 000d) Study Instance UID                  UI: 1.3.6.1.4.1.11129.5.5.112507010803284478207522016832191866964708
(0020, 000e) Series Instance UID                 UI: 1.3.6.1.4.1.11129.5.5.112630850362182468372440828755218293352329
(0028, 0002) Samples per Pixel                   US: 1
(0028, 0004) Photometric Interpretation          CS: 'MONOCHROME2'